<a href="https://colab.research.google.com/github/javihm77/CapstoneProjectImperialCollege/blob/main/YouTube_comments_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import googleapiclient.discovery
import pandas as pd
from google.colab import userdata
import time

In [ ]:
video_ids = ["bi9Mtc-VU1Q", "6PuLDJ2g7QU","0ClDGAIlHNI","wzoOTgcVcUs"
              ,"w3pb1I6K5fU","avsZv8rS6No","n00jNpEqePw","bGkH_eBLEHI"
              ,"zglqcGY2z0M","C9uL_ZUdYI0","-VIxZG1ujaQ","aIR6Mt90Yfc"
              ,"1f9fZ_fufvE","9s10p00BcK0","7pyJt076mOw","Gdy6OliW6Rc","IGVFTK8f91A"
              ,"PNdNDcA-GUA","QC4h5rZoeSQ","ZWhYLwBoO6I","nT3Prnx-GQk","WyRUQiRUfsQ"
              ,"DAEhzrKHYtQ","oweq8jr9INs","dN87pXGcrxA","i6AD9UUNu8s","2r-ivDXDqio"
              ,"ij6EBM_eX0A","Kg3gZjVcpR0","bOFaDiQDiuU","1Q3mN6BmeQk","UMMTwDjohAI"
              ,"lg5vMRb51YI","fAq03mGKTRQ","E8lfXu6PyNY","WBXb3DhIqxM","65R6YTJY1Ss"
              ,"kT2ls-kN-sg","ahXyjTP9dso","SAwSWfp401I","4NNTUffXD-U","BWg_gthAc3o"
              ,"uGEyCcAJpuE","eyquilNeMrI","aGcvoQfcKnQ","cpQfbqjKyt0","r7fRWYC92q8"
              ,"78N5XlZoc_Q","8J98ehjq92I","zec0yICf7l0","PV0jri85yyo","pz5QsklC640"
              ,"aPFvKehZ44s","cWSJ59C7fgs","mKYOYGmGwFs","6jo6QIhFWdc","_FujLTTk3jA"
              ,"sQUS7Sy_jT8","i0AgHX7ES8w","s4nSmmnGLN0","dktgFgYm40s","myJwzTRR3xM"
              ,"T7iS8B4MteM","aH0dy9rjBLI","avdUJiOxRro","za066SxcnUY","xAeHikredIg"
              ,"zJfh5ijiUbA","JXl-EaAfQ-c","Az_iX4VnVsQ","VB0mvudKjIQ","J2aV1GLt0SI"
              ,"W_DJTXPnXok","qWRN_-ADhdE","rikjN6xqiRY","aLH10eZTYe4","vEADbKZ40NA"]

In [ ]:
Youtube_api_secret_name = 'YouTube'  # @param {type: "string"}

try:
  api_key=userdata.get(Youtube_api_secret_name)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {Youtube_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {Youtube_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {Youtube_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {Youtube_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

In [ ]:
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = api_key

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)


def getcomments(video):
  request = youtube.commentThreads().list(
      part="snippet",
      videoId=video,
      maxResults=1000
  )

  comments = []

  # Execute the request.
  try:
    response = request.execute()

    # Get the comments from the response.
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        public = item['snippet']['isPublic']
        # Create a dictionary for each comment
        comment_dict = {
            'author': comment['authorDisplayName'],
            'updated_at': comment['publishedAt'],
            'like_count': comment['likeCount'],
            'text': comment['textOriginal'],
            'video_id': comment['videoId'],
            'public': public
        }
        comments.append(comment_dict)  # Append the dictionary to the list

    while True:  # Use True for infinite loop
        try:
            nextPageToken = response['nextPageToken']
            # Create a new request object with the next page token.
            nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=1000, pageToken=nextPageToken)
            # Execute the next request.
            response = nextRequest.execute()  # Update response for the next page
            # Get the comments from the next response.
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                public = item['snippet']['isPublic']
                comment_dict = {
                    'author': comment['authorDisplayName'],
                    'updated_at': comment['publishedAt'],
                    'like_count': comment['likeCount'],
                    'text': comment['textOriginal'],
                    'video_id': comment['videoId'],
                    'public': public
                }
                comments.append(comment_dict)
        except KeyError:
            break  # Exit the loop when 'nextPageToken' is not found

  except Exception as e:
    print(f"Error fetching comments: {e}")
    time.sleep(60)  # Add a delay to avoid rate limiting

  return comments


In [ ]:
def get_video_details(youtube, video_id):
        response = youtube.videos().list(
            part="snippet,statistics",
            id=video_id
        ).execute()

        if not response['items']:
            return None

        video_data = response['items'][0]
        return {
            'video title': video_data['snippet']['title'],
            'video published_date': video_data['snippet']['publishedAt'],
            'video view_count': video_data['statistics']['viewCount'],
            'video like_count': video_data['statistics'].get('likeCount', 0),
            'video dislike_count': video_data['statistics'].get('dislikeCount', 0),
            'video comment_count': video_data['statistics']['commentCount']
        }


In [ ]:
df = getcomments('-VIxZG1ujaQ')
df[0:4]

[{'author': '@pinuccia9269',
  'updated_at': '2024-08-11T21:45:30Z',
  'like_count': 0,
  'text': 'Por que enojan. Que dejen que el Mp investigué  sino tienen cola que les tomen. Cual es el temor',
  'video_id': '-VIxZG1ujaQ',
  'public': True},
 {'author': '@gdegt',
  'updated_at': '2024-07-29T09:06:37Z',
  'like_count': 0,
  'text': 'el mal manejo de la depresion  de ben  transmitida  TE ESTA AFECTANDO QUIQUE\n  ojo  ben es depresivo',
  'video_id': '-VIxZG1ujaQ',
  'public': True},
 {'author': '@gdegt',
  'updated_at': '2024-07-29T09:05:19Z',
  'like_count': 0,
  'text': 'quique salite de este canal de chin\n YA ESTE CUETE ESTA MA QUE QUEMADO  y  NADIE LE PONE COCO ,,,,, este cuate de ben  SOLO SOS DEPRIME\nquique salite de esta onda',
  'video_id': '-VIxZG1ujaQ',
  'public': True},
 {'author': '@lorenamelgar4210',
  'updated_at': '2024-07-26T02:53:43Z',
  'like_count': 0,
  'text': 'yo también estoy por las Asias y me toca verlos en doferido 😊 pero nunca me los pierdo, si no me eno

In [ ]:
all_data = []

for video_id in video_ids:
  print(video_id)
  comments = getcomments(video_id)
  video_info = get_video_details(youtube, video_id)

  for comment in comments:
    comment.update(video_info)
  all_data.extend(comments)

df = pd.DataFrame(all_data)
df = df.sort_values(by=['video published_date', 'updated_at'], ascending=[True,True])


bi9Mtc-VU1Q
6PuLDJ2g7QU
0ClDGAIlHNI
wzoOTgcVcUs
w3pb1I6K5fU
avsZv8rS6No
n00jNpEqePw
bGkH_eBLEHI
zglqcGY2z0M
C9uL_ZUdYI0
-VIxZG1ujaQ
aIR6Mt90Yfc
1f9fZ_fufvE
9s10p00BcK0
7pyJt076mOw
Gdy6OliW6Rc
IGVFTK8f91A
PNdNDcA-GUA
QC4h5rZoeSQ
ZWhYLwBoO6I
nT3Prnx-GQk
WyRUQiRUfsQ
DAEhzrKHYtQ
oweq8jr9INs
dN87pXGcrxA
i6AD9UUNu8s
2r-ivDXDqio
ij6EBM_eX0A
Kg3gZjVcpR0
bOFaDiQDiuU
1Q3mN6BmeQk
UMMTwDjohAI
lg5vMRb51YI
fAq03mGKTRQ
E8lfXu6PyNY
WBXb3DhIqxM
65R6YTJY1Ss
kT2ls-kN-sg
ahXyjTP9dso
SAwSWfp401I
4NNTUffXD-U
BWg_gthAc3o
uGEyCcAJpuE
eyquilNeMrI
aGcvoQfcKnQ
cpQfbqjKyt0
r7fRWYC92q8
78N5XlZoc_Q
8J98ehjq92I
zec0yICf7l0
PV0jri85yyo
pz5QsklC640
aPFvKehZ44s
cWSJ59C7fgs
mKYOYGmGwFs
6jo6QIhFWdc
_FujLTTk3jA
sQUS7Sy_jT8
i0AgHX7ES8w
s4nSmmnGLN0
dktgFgYm40s
myJwzTRR3xM
T7iS8B4MteM
aH0dy9rjBLI


In [ ]:
df.head(4)

,author,updated_at,like_count,text,video_id,public,video title,video published_date,video view_count,video like_count,video dislike_count,video comment_count
625,@carolnavarro5510,2024-05-30T20:28:22Z,3,Muy buena idea😅😅 el negocio de los stickers 😂,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-30T19:17:19Z,3580,181,0,6
624,@delilahcurtiscabral4956,2024-05-30T21:13:57Z,1,Tanto Dineroooo!!! Por eso la mayoría del cír...,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-30T19:17:19Z,3580,181,0,6
623,@aurapalencia2273,2024-05-30T23:54:28Z,1,Excelente exposición del tema,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-30T19:17:19Z,3580,181,0,6
622,@OscarLopezSandoval-mp2wp,2024-05-31T00:12:57Z,0,Quique... donde estan los delfines ?,zec0yICf7l0,True,LA OTRA CORTE CELESTIAL - PFNSE,2024-05-30T19:17:19Z,3580,181,0,6


In [ ]:
df.tail(4)

,author,updated_at,like_count,text,video_id,public,video title,video published_date,video view_count,video like_count,video dislike_count,video comment_count
826,@aguilar4254,2024-09-03T21:51:10Z,0,Entonces que va hacer con los buses y túc túc ...,aH0dy9rjBLI,True,EL QUE ES VUELVE,2024-09-03T19:00:34Z,2413,143,0,10
825,@irmabrenes2412,2024-09-03T21:58:10Z,1,"Leasing=arrendamiento, por si se le está olvid...",aH0dy9rjBLI,True,EL QUE ES VUELVE,2024-09-03T19:00:34Z,2413,143,0,10
824,@cesarmejiaochoa4845,2024-09-03T22:08:31Z,0,El famoso leasing,aH0dy9rjBLI,True,EL QUE ES VUELVE,2024-09-03T19:00:34Z,2413,143,0,10
823,@rodilmartinez9958,2024-09-03T22:32:52Z,0,Si puede indicar después de los 36 meses quien...,aH0dy9rjBLI,True,EL QUE ES VUELVE,2024-09-03T19:00:34Z,2413,143,0,10


In [ ]:
df.shape

(832, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 832 entries, 625 to 823
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   author                832 non-null    object
 1   updated_at            832 non-null    object
 2   like_count            832 non-null    int64 
 3   text                  832 non-null    object
 4   video_id              832 non-null    object
 5   public                832 non-null    bool  
 6   video title           832 non-null    object
 7   video published_date  832 non-null    object
 8   video view_count      832 non-null    object
 9   video like_count      832 non-null    object
 10  video dislike_count   832 non-null    int64 
 11  video comment_count   832 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 111.1+ KB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.to_csv("/content/drive/MyDrive/Imperial College/Capstone Project/raw_youtube_comments.csv", index=True)